# 함수형 기능 소개

대부분의 함수형 프로그래밍 기능은 파이썬에 이미 일급 계층으로 들어있다. 함수형 파이썬을 작성하는 목표는 명령형 기법에서 가능한 멀리 벗어나는 것이다.

## 일급 계층 함수
함수형 프로그래밍은 간결하고 표현력이 높은데 이를 달성하려면 함수를 인자로 제공하고, 다른 함수가 사용할 수 있도록 함수를 반환하는 것이다.
이러한 사용이 가능하려면 함수가 실행 시점의 환경에서 일급 계층이어야만 한다. C와 같은 프로그래밍 언어에서는 함수가 실행 시점의 객체가 아니지만 파이썬은 def문에 의해 생성되는 객체이며, 다른 파이썬 함수에서 이를 조작할 수 있고, 호출 가능(callable) 객체로 정의하거나 lambda를 변수에 대입하여 만들 수 있다.

In [3]:
def example(a, b, **kw):
    return a*b

type(example)
example.__code__.co_varnames
example.__code__.co_argcount

2

__code__ 객체는 함수 객체 자신을 애트리뷰트와 연관시켜준다. 함수가 일급 계층 객체이고, 다른 일반적인 객체와 마찬가지로 조작이 가능하다는 것을 볼 수 있다.

## 순수 함수

함수형 프로그래밍 설계에서 사용하는 함수는 부수 효과로 인해 발생할 수 있는 혼동이 없어야 한다. 순수 함수를 사용하면 평가 순서를 바꿔 최적화할 수 있는 여지가 생기지만 가장 큰 이익은 순수 함수가 개념적으로 훨씬 단순하며, 테스트하기 쉽다는 점으로부터 얻을 수 있다.

순수 함수를 파이썬으로 작성하려면 지역 상태만을 사용하는 코드를 작성해야 한다. 만일 정말 순수 함수 규칙을 따르고 있는지를 자신할 수 없다면 dis 모듈을 사용해 주어진 함수를 컴파일한 코드인 __code__.co_code 부분에 전역 참조가 있는지 살펴보아야 한다. dis 모듈은 내부 클로저의 사용을 보고할 수 있다. 

파이썬 lamdba는 순수 함수다. 람다는 권장하진 않지만 순수 함수를 만드는 것은 가능하다.

In [5]:
mersenne = lambda x:2**x-1
mersenne(17)

131071

람다 내부에 대입문을 사용할 수 없기 때문에 람다는 항상 순수 함수이며, 함수형 프로그래밍에 적합하다.

## 고차 함수

고차 함수를 사용하면 간결하고 이해하기 쉬운 프로그램을 만들 수 있다. 함수를 인자로 받아들이거나 함수를 값으로 반환하는 함수도 있다.
파이썬의 max()에 함수를 인자로 제공하면 max()가 작동하는 방식을 바꿀 수 있다

In [6]:
year_cheese =[ (4000, 20), (3000, 30), (2000, 40) ]
max(year_cheese)

(4000, 20)

In [7]:
max(year_cheese, key =lambda x:x[1])

(2000, 40)

## 변경 불가능한 데이터

계산 상태를 추적할 때는 변수를 사용하지 않기 때문에 초점을 변경 불가능한 객체에 맞춰야 한다. 우리는 tuple과 namedtuple을 폭넓게 사용해 변경 불가능한 복잡한 데이터를 제공할 것이다.

함수형 프로그래밍에서는 단지 상태가 있는 객체가 필요하지 않아서 객체를 사용하지 않는다. callable 객체를 정의해야 할 경우가 있다. 그러한 객체는 밀접하게 연관된 함수에 대한 이름 공간을 제공하는 깔끔한 방법이며, 성정 가능성도 다양하게 제공하기 때문이다.

불변 객체를 사용하면 특히 잘 작동하는 일반적인 디자인 패턴을 살표보면 wrapper 함수가 있다. 튜플의 리스트는 매우 자주 쓰이는 데이터 구조인데 튜플의 리스트를 처리할 때는 보통 다음 두 가지 방법 중 하나를 선택한다.

1. 고차 함수 max(year_cheese, key=lambda x:x[1])
2. 감싸고-처리하고-풀기 패턴 max(map(lambda x:(x[1], x), year_cheese))

In [33]:
max(map(lambda x:(x[1], x), year_cheese))

(40, (2000, 40))

In [35]:
_[1] # 첨자 연산은 max() 함수가 선택한 2-튜플의 두 번째 원소를 가져온다.

(2000, 40)

In [36]:
snd = lambda x:x[1]
snd( max(map(lambda x:(x[1], x), year_cheese)))

(2000, 40)

## 엄격한 평가와 엄격하지 않은 평가

함수형 프로그래밍은 부분적으로 어떤 계산을 그 결과가 정말 필요할 때까지 유예하는 방식을 활용하는 것으로 인해 효율성을 얻는다. 이러잔 지연 또는 엄격하지 않은 평가라는 아이디어는 매우 유용하다. 

파이썬에서 논리 연산자인 and, or나 if-then-else는 모두 엄격하지 않다. 결과값을 계산하는기 위해 항상 모든 인자를 평가하지는 않기 때문이다. 

아래의 명령행 세션은 이러한 연산자의 엄격하지 않은 특성을 보여준다.

In [40]:
0 and print('right')

0

In [41]:
True and print('right')

right


메서드 함수는 클래스를 정의할 때 엄격하게 순서대로 정의된다. 클래스 정의의 경우 메서드 함수들은 딕셔너리에 수집되며, 생성된 다음에는 순서를 유지하지 않는다. 이름이 같은 메서드를 두 가지 정의하면, 엄격한 정의 순서에 따라 두 번째 메서드만 살아남는다.

하지만 파이썬의 제너레이터 식이나 함수는 지연 계산 대상이다. 이러한 식은 즉시 모든 가능한 값을 만들어 내지 않는다. 계산 과정을 명시적으로 로그에 남기지 않으면 이러한 특성을 살펴보는 것은 쉽지 않다. 다음은 range() 함수의 다른 버전으로, 내부적으로 부수 효과를 사용해 생성 중인 수를 표시하는 함수다.

In [44]:
def numbers():
    for i in range(2004):
        print("=", i)
        yield i

In [50]:
def sum_to(n):
    sum = 0
    for i in numbers():
        if i == n: break
        sum += i
    return sum

In [51]:
sum_to(5)

= 0
= 1
= 2
= 3
= 4
= 5


10

파이썬의 제네레이터 함수에는 단순한 함수형 프로그래밍에 사용하기에는 조금 이상한 특징이 존재한다. 특히, 파이썬에서는 제네레이터를 한 번만 사용할 수 있다. 따라서 지연 계산하는 파이썬 제네레이터 식을 사용하는 경우에는 조심해야 할 필요가 있다.

## 명시적 루프 상태 대신 재귀 사용

함수형 프로그램은 루프에 의존하지 않으며, 루프의 상태를 추적하는 데 따른 부가 비용도 없다. 